# Start Evaluation

In [1]:
import warnings
import pickle
import datetime
import time

import csv
import numpy as np
import pandas as pd 

import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [3]:
def draw_line_and_calculate_gap(image, start_point, end_point):
    if start_point and end_point:
        x1, y1 = start_point.x * image.shape[1], start_point.y * image.shape[0]
        x2, y2 = end_point.x * image.shape[1], end_point.y * image.shape[0]
        
        # Calculate normalized coordinates
        gap_x = (x2 - x1) / image.shape[1]
        gap_y = (y2 - y1) / image.shape[0]

        # Draw line
        x1, y1 = int(x1), int(y1)
        x2, y2 = int(x2), int(y2)
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 3)

        gap_landmark = landmark_pb2.NormalizedLandmark()
        gap_landmark.x = gap_x
        gap_landmark.y = gap_y

        return gap_landmark
    else:
        return None

In [4]:
with open('../Model/svm_mpgap.pkl', 'rb') as f:
    model = pickle.load(f)

model

c:\Users\nawfa\anaconda3\envs\cuda_12_1_clone\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\nawfa\anaconda3\envs\cuda_12_1_clone\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\nawfa\anaconda3\envs\cuda_12_1_clone\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.3.1 when us

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(probability=True))])

# Code

In [5]:
def evaluasi_model(file, pose_limit, poses):
    cap = cv2.VideoCapture(f'Scenario/{file}')

    counter = 0
    recent_pose = None

    pose_sequence = poses
    pose_detected = []
    condition = []

    print(pose_limit)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
        while cap.isOpened():
            ret, frame = cap.read()
        
            # Recolor Feed
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False        
        
            # Make Detections
            results = holistic.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True   
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    

            # Get specific landmarks
            nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
            wrist_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_WRIST]
            elbow_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_ELBOW]
            wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]
            elbow_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_ELBOW]
        
            # Create a custom landmark list to show
            show_landmark_list = landmark_pb2.NormalizedLandmarkList()
            show_landmark_list.landmark.extend([nose, wrist_l, wrist_r, elbow_l, elbow_r])
        
            # Draw landmarks
            for landmark in show_landmark_list.landmark:
                x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
                cv2.circle(image, (x, y), 5, (255, 0, 0), -1)
        
            # Drawing line and calculating gap for left wrist
            gap_nose_left = draw_line_and_calculate_gap(image, nose, wrist_l)

            # Drawing line and calculating gap for right wrist
            gap_nose_right = draw_line_and_calculate_gap(image, nose, wrist_r)

            gap_hand_left = draw_line_and_calculate_gap(image, elbow_l, wrist_l)
            gap_hand_right = draw_line_and_calculate_gap(image, elbow_r, wrist_r)


            new_lm = landmark_pb2.NormalizedLandmarkList()
            new_lm.landmark.extend([wrist_l, wrist_r, elbow_l, elbow_r, gap_nose_right, gap_nose_left])
            # ===================================================== #

            # Export coordinates
            try:
                # Extract Pose landmarks
                pose = new_lm.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
        
                # Make Detections
                X = pd.DataFrame([pose_row])

                # Ignore scikit-learn warnings about feature names
                warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]
            
            
                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
                pose_percentage = round(body_language_prob[np.argmax(body_language_prob)],2)
                if pose_percentage >= 0.70:
                    # Display Class
                    cv2.putText(image, 'CLASS'
                                , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(image, body_language_class.split(' ')[0]
                                , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
                    # Display Probability
                    cv2.putText(image, 'PROB'
                                , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                    cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                                , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                                
                    if body_language_class != recent_pose:
                        recent_pose = body_language_class.split(' ')[0]
                        
                        condition.append(poses[0] == recent_pose)
                        pose_detected.append(recent_pose)
                        
                        
                        print(recent_pose, counter)
                        counter += 1

                    if counter == pose_limit:
                        cap.release()
                        cv2.destroyAllWindows()
                        return pose_detected, condition
                
            except Exception as e:
                print(f"Error during prediction: {e}")
                        
            cv2.imshow('Raw Webcam Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows() 

# READ SCENARIO CSV

In [7]:
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')    

with open('Scenario/scenario.csv', 'r') as csvfile, open(f'Tested/{timestamp}.csv', 'w', newline='') as outfile:
    reader = csv.reader(csvfile)
    data = list(reader)
    writer = csv.writer(outfile)

    for row in data:
        scenario = row[0]
        file = row[1]
        pose = row[2:]

        evaluated = evaluasi_model(file, len(pose)-1, pose)
        print(evaluated)
    
        modified_row = row + ['detected'] + evaluated[0] + ['condition'] + evaluated[1]
        writer.writerow(modified_row)    

5
idle 0
guard 1
jab 2
guard 3
jab 4
(['idle', 'guard', 'jab', 'guard', 'jab'], [False, False, True, False, True])
